# 1) Extraindo os dados dos arquivos PDF
> Seção introdutória (markdown) que anuncia a etapa de extração de textos a partir de PDFs.

---

## Bloco 1 — Instalações de dependências
**O que faz:** instala bibliotecas para leitura de PDF:

- pypdf — parser de PDFs (lê estrutura, páginas).

- pdfminer.six — extrator de texto de PDF (converte PDF→texto).

**Por quê:** garantir ambiente reprodutível com versões fixadas (evita “quebras” ao longo do tempo).

**Saída:** nenhuma variável; apenas instala pacotes no ambiente.

In [4]:
%pip install pdfplumber==0.11.0 regex==2024.9.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 44.5 MB/s eta 0:00:00
  Attempting uninstall: regex
    Found existing installation: regex 2025.10.23
    Uninstalling regex-2025.10.23:
      Successfully uninstalled regex-2025.10.23
   ━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [pypdfium2]  WARNING: The script pypdfium2 is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20240706━━━━━━━━ 1/4 [pypdfium2]
    Uninstalling pdfminer.six-20240706:━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [pypdfium2]
      Successfully uninstalled pdfminer.six-20240706━━━━━━━━━━ 1/4 [pypdfium2]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━ 3/4 [pdf

## Bloco 2 — Varredura e cache de parágrafos por pasta/ano

**O que faz:** percorre a **raiz** com PDFs, filtra por **anos** desejados, extrai parágrafos de cada documento (via `processar_pdf`) e salva/recupera **cache** em `*.pkl` (arquivo binário do pandas) para acelerar novas execuções.

**Parâmetros (principais):**

* `raiz` (caminho da pasta): diretório onde estão os PDFs.
* `min_palavras` (limiar de qualidade): descarta/funde parágrafos muito curtos.
* `anos` (filtro temporal): limita os PDFs pelos anos no nome do arquivo (padrão `{2022, 2023, 2024, 2025}`).
* `cache_dir` (cache em disco): pasta dos `*.pkl`.
* `rebuild_cache` (reprocessar?): se `True`, ignora cache e reconstrói.

**Conceitos rápidos:**

* **Cache**: armazenamento intermediário de resultados para evitar reprocessar tudo a cada execução.
* **YYYYMM**: carimbo de ano+mês (ex.: `202312`).

**Saída:** `DataFrame` com colunas `["doc","dt","pid","texto","n_palavras"]`, ordenado.


---

In [ ]:
# -*- coding: utf-8 -*-
import re, unicodedata, warnings
from pathlib import Path
from datetime import datetime
from typing import List, Set, Tuple, Optional
import pandas as pd
from pypdf import PdfReader

# pdfplumber (preferencial)
try:
    import pdfplumber
except Exception:
    pdfplumber = None

# pdfminer (fallback)
try:
    from pdfminer.high_level import extract_text as pdfminer_extract_text
except Exception:
    pdfminer_extract_text = None

# ------------------ REGEX & HEURÍSTICAS ------------------

PAGE_RE = re.compile(r'^\s*(p[aá]gina)\s+\d+(\s+de\s+\d+)?\s*$', re.I)
ONLY_DIGITS_RE = re.compile(r'^\s*\d+\s*$')
CAPTION_RE = re.compile(r'^\s*(figura|gr[aá]fico|tabela)\s*\d+[\.:].*$', re.I)
NOTE_PREFIX_RE = re.compile(r'^\s*(nota[s]?|obs\.?|observa[cç][aã]o|fonte|elabora[cç][aã]o)\s*[:\-–].*$', re.I)
FOOTNOTE_MARK_RE = re.compile(r'\[\d+\]')                  # [1], [23], ...
SUPERSCRIPTS_RE = re.compile(r'[\u00B9\u00B2\u00B3\u2070-\u2079]+')  # ¹²³⁴…
STOP_AFTER_HEADINGS_RE = re.compile(r'^\s*(refer[eê]ncias|anexos?|gloss[aá]rio)\b.*$', re.I | re.M)

HEADER_HINTS = (
    "banco central do brasil",
    "comitê de política monetária",
    "relatório de estabilidade financeira",
    "ata do copom",
    "relatório de estatística",
    "política monetária",
)

DOC_TYPE_CONFIG = {
    "ata":    {"stop_after": STOP_AFTER_HEADINGS_RE},
    "ref":    {"stop_after": STOP_AFTER_HEADINGS_RE},
    "mensal": {"stop_after": STOP_AFTER_HEADINGS_RE},
}

def normalize_line(s: str) -> str:
    s = unicodedata.normalize("NFC", s).strip()
    s = re.sub(r'\s+', ' ', s)
    return s

def is_short_heading(line: str, max_words: int = 8) -> bool:
    l = normalize_line(line)
    if not l:
        return False
    words = l.split()
    if len(words) > max_words:
        return False
    letters = [ch for ch in l if ch.isalpha()]
    if not letters:
        return False
    upper_ratio = sum(ch.isupper() for ch in letters) / len(letters)
    return upper_ratio >= 0.8

def join_hyphenated(text: str) -> str:
    # junta palavra-\nseguinte -> palavraseguinte
    return re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)

def normalize_whitespace(text: str) -> str:
    text = text.replace("\r", "\n")
    text = re.sub(r'\n{3,}', '\n\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)  # quebra simples -> espaço
    text = re.sub(r' {2,}', ' ', text)
    text = re.sub(r'\s+([,.;:!?])', r'\1', text)
    text = re.sub(r'\n{3,}', '\n\n', text).strip()
    return text

def detect_repeating_edge_lines(pages_lines: List[List[str]],
                                top_n: int = 2, bottom_n: int = 2,
                                threshold: float = 0.5) -> Set[str]:
    from collections import Counter
    cand = Counter()
    total = max(1, len(pages_lines))
    for lines in pages_lines:
        lines_norm = [normalize_line(x) for x in lines if x.strip()]
        top = lines_norm[:top_n]
        bottom = lines_norm[-bottom_n:] if bottom_n else []
        for item in set(top + bottom):
            if item:
                cand[item] += 1
    rep = {s for s, c in cand.items() if c / total >= threshold}
    rep |= {s for s in cand if any(h in s.lower() for h in HEADER_HINTS)}
    return rep

def remove_edge_lines(lines: List[str], repetitive: Set[str]) -> List[str]:
    out = []
    for ln in lines:
        n = normalize_line(ln)
        if not n:
            continue
        if n in repetitive:
            continue
        if PAGE_RE.match(n) or ONLY_DIGITS_RE.match(n):
            continue
        out.append(ln)
    return out

def clean_page_text(raw: str, repetitive: Set[str]) -> str:
    lines = raw.splitlines()
    lines = remove_edge_lines(lines, repetitive)

    kept = []
    for ln in lines:
        l = normalize_line(ln)
        if not l:
            continue
        if is_short_heading(l):
            continue
        if NOTE_PREFIX_RE.match(l) or CAPTION_RE.match(l):
            continue
        l = FOOTNOTE_MARK_RE.sub('', l)
        l = SUPERSCRIPTS_RE.sub('', l)
        kept.append(l)
    return "\n".join(kept).strip()

def apply_stop_after(text: str, doc_type: str) -> str:
    pat = DOC_TYPE_CONFIG.get(doc_type, {}).get("stop_after")
    if not pat:
        return text
    m = pat.search(text)
    return text[:m.start()].rstrip() if m else text

def guess_doc_type(name: str) -> str:
    s = name.lower()
    if "ata" in s and "copom" in s: return "ata"
    if "estabilidade" in s or "ref" in s: return "ref"
    return "mensal"  # Relatório de estatística e política monetária

# ------------------ EXTRAÇÃO (página a página) ------------------

def extract_pages_text(path_pdf: Path) -> List[str]:
    pages: List[str] = []
    # 1) pdfplumber (preferido: ciente de layout)
    if pdfplumber is not None:
        try:
            with pdfplumber.open(str(path_pdf)) as pdf:
                for page in pdf.pages:
                    t = page.extract_text() or ""
                    pages.append(t)
        except Exception:
            pages = []
    # 2) pypdf (fallback)
    if not pages:
        try:
            reader = PdfReader(str(path_pdf))
            pages = [(p.extract_text() or "") for p in reader.pages]
        except Exception:
            pages = []
    # 3) pdfminer (fallback final)
    if (not pages or not any(x.strip() for x in pages)) and pdfminer_extract_text:
        try:
            txt = pdfminer_extract_text(str(path_pdf)) or ""
            # pdfminer separa páginas por \f
            pages = [p for p in re.split(r'\f+', txt)]
        except Exception:
            pages = []
    return pages

def clean_pdf_text(path_pdf: Path, doc_type: str) -> Tuple[str, Set[str], int]:
    raw_pages = extract_pages_text(path_pdf)
    if not raw_pages or not any(p.strip() for p in raw_pages):
        return "", set(), 0

    pages_lines = [p.splitlines() for p in raw_pages]
    repetitive = detect_repeating_edge_lines(pages_lines, top_n=2, bottom_n=2, threshold=0.5)
    cleaned_pages = [clean_page_text(p, repetitive) for p in raw_pages]
    text = "\n\n".join(p for p in cleaned_pages if p.strip())

    text = join_hyphenated(text)
    text = normalize_whitespace(text)
    text = apply_stop_after(text, doc_type)
    return text, repetitive, len(raw_pages)

# ------------------ PARÁGRAFOS E DATA ------------------

def quebrar_paragrafos(txt: str, min_palavras: int = 8) -> list[str]:
    brutos = [p.strip() for p in re.split(r"\n\s*\n", txt) if p.strip()]
    pars = []
    for p in brutos:
        p = " ".join([l.strip() for l in p.split("\n") if l.strip()])
        if len(p.split()) >= min_palavras:
            pars.append(p)
    return pars

def data_do_arquivo(path_pdf: Path) -> tuple[str, pd.Timestamp]:
    """
    Detecta 202201 (YYYYMM) ou 20220131 (YYYYMMDD) em qualquer lugar do nome.
    Retorna:
      - yyyymm (sempre 6 dígitos)
      - dt (Timestamp; se tiver dia usa-o, senão 1º dia do mês)
    """
    name = path_pdf.stem
    m8 = re.search(r'(?<!\d)(\d{8})(?!\d)', name)
    m6 = re.search(r'(?<!\d)(\d{6})(?!\d)', name)
    if m8:
        s = m8.group(1)
        dt = datetime.strptime(s, "%Y%m%d")
        return s[:6], pd.Timestamp(dt.year, dt.month, dt.day)
    if m6:
        s = m6.group(1)
        dt = datetime.strptime(s, "%Y%m")
        return s, pd.Timestamp(dt.year, dt.month, 1)
    raise ValueError(f"Não encontrei data no nome do arquivo: {name}")

# ------------------ PROCESSAMENTO POR DOC ------------------

def processar_pdf(path_pdf: Path, min_palavras: int = 8,
                  doc_type: Optional[str] = None,
                  yyyymm_override: Optional[str] = None) -> pd.DataFrame:
    try:
        yyyymm, dt = data_do_arquivo(path_pdf)
    except Exception:
        # Mantém compatibilidade com casos sem data — retorna vazio
        return pd.DataFrame(columns=["doc","dt","pid","texto","n_palavras"])

    if yyyymm_override:
        yyyymm = yyyymm_override

    doc_type = doc_type or guess_doc_type(path_pdf.name)
    # doc_id diferencia mês por tipo (evita colisões de cache/duplicatas)
    doc_id = f"{yyyymm}_{doc_type}"

    text, _removed, _n_pages = clean_pdf_text(path_pdf, doc_type)
    if not text.strip():
        return pd.DataFrame(columns=["doc","dt","pid","texto","n_palavras"])

    pars = quebrar_paragrafos(text, min_palavras=min_palavras)
    rows = [{"doc": doc_id, "dt": dt, "pid": i, "texto": p, "n_palavras": len(p.split())}
            for i, p in enumerate(pars, 1)]
    return pd.DataFrame(rows)

# ------------------ VARREDURA, CACHE E JUNÇÃO ------------------

def processar_raiz(raiz: str,
                   min_palavras: int = 8,
                   anos: set[int] | None = {2022, 2023, 2024, 2025},
                   cache_dir: str = "out/estatisticas_paragrafos",
                   rebuild_cache: bool = False) -> pd.DataFrame:
    """
    Saída: DataFrame com colunas ["doc","dt","pid","texto","n_palavras"], ordenado.
    - "doc" agora é "YYYYMM_tipo" (ex.: "202312_ata", "202406_ref", "202411_mensal")
      para evitar colisões quando há mais de um PDF no mesmo mês.
    """
    raiz = Path(raiz)
    cache = Path(cache_dir)
    cache.mkdir(parents=True, exist_ok=True)

    dfs, vistos = [], set()

    for pdf in sorted(raiz.rglob("*.pdf")):
        name = pdf.stem

        # detecta 8d -> 6d; ou só 6d
        m8 = re.search(r'(?<!\d)(\d{8})(?!\d)', name)
        m6 = re.search(r'(?<!\d)(\d{6})(?!\d)', name)
        if m8:
            yyyymm = m8.group(1)[:6]
        elif m6:
            yyyymm = m6.group(1)
        else:
            continue

        ano = int(yyyymm[:4])
        if anos is not None and ano not in anos:
            continue

        tipo = guess_doc_type(name)
        doc_key = f"{yyyymm}_{tipo}"

        if doc_key in vistos:
            continue

        pkl_path = cache / f"{doc_key}.pkl"
        if (not rebuild_cache) and pkl_path.exists():
            try:
                df = pd.read_pickle(pkl_path)
            except Exception:
                df = processar_pdf(pdf, min_palavras=min_palavras, doc_type=tipo, yyyymm_override=yyyymm)
                df.to_pickle(pkl_path)
        else:
            df = processar_pdf(pdf, min_palavras=min_palavras, doc_type=tipo, yyyymm_override=yyyymm)
            df.to_pickle(pkl_path)

        if not df.empty:
            dfs.append(df)
            vistos.add(doc_key)

    if not dfs:
        return pd.DataFrame(columns=["doc","dt","pid","texto","n_palavras"])

    out = pd.concat(dfs, ignore_index=True)
    return out.sort_values(["dt","doc","pid"]).reset_index(drop=True)

# Alias para nome canônico na base consolidada
TIPO_ALIAS_OUT = {
    "mensal": "estm",
    "ata":    "copom",
    "ref":    "ref",
}

def doc_canonico(doc_str: str) -> str:
    """
    Recebe 'YYYYMM_tipo' (ex.: '202403_mensal') e devolve
    'YYYYMM_estm|copom|ref' conforme o padrão solicitado.
    """
    yyyymm = doc_str[:6]
    tipo = doc_str.rsplit("_", 1)[-1]
    return f"{yyyymm}_{TIPO_ALIAS_OUT.get(tipo, tipo)}"

# === CONSOLIDAÇÃO POR DOCUMENTO (1 linha = 1 PDF) ===
def montar_docs(df_paragrafos: pd.DataFrame) -> pd.DataFrame:
    base = df_paragrafos.sort_values(["doc", "pid"]).copy()
    df_docs = (
        base.groupby("doc")
            .agg(
                dt=("dt", "first"),
                texto=("texto", lambda s: "\n\n".join(s)),
                n_paragrafos=("pid", "count"),
                n_palavras=("n_palavras", "sum"),
            )
            .reset_index()
    )

    # metadados
    df_docs["tipo"] = df_docs["doc"].str.rsplit("_", n=1).str[-1]  # ata/ref/mensal
    df_docs["yyyymm"] = df_docs["doc"].str[:6]

    # nome canônico solicitado
    df_docs = df_docs.rename(columns={"doc": "doc_src"})
    df_docs["doc"] = df_docs["doc_src"].apply(doc_canonico)

    # reordenar colunas (opcional)
    cols = ["doc", "doc_src", "yyyymm", "tipo", "dt", "n_paragrafos", "n_palavras", "texto"]
    return df_docs[cols]


**O que faz:**

1. Executa a coleta/extração de parágrafos e guarda em `df_paragrafos`.
2. Exporta para `paragrafos.csv`.

**Por quê:** persistir em CSV facilita auditoria e uso em outras ferramentas.

**Saída:** arquivo `paragrafos.csv` no diretório atual.

---

In [ ]:
df_paragrafos = processar_raiz("data/estatisticas", anos={2022, 2023, 2024, 2025})
df_docs = montar_docs(df_paragrafos)
df_docs



,doc,doc_src,yyyymm,tipo,dt,n_paragrafos,n_palavras,texto,sentimento
0,202201_estm,202201_mensal,202201,mensal,2022-01-01,4,1536,1. Crédito ampliado ao setor não financeiro Em...,0.0
1,202202_estm,202202_mensal,202202,mensal,2022-02-01,5,1686,1. Crédito ampliado ao setor não financeiro Em...,0.0
2,202203_estm,202203_mensal,202203,mensal,2022-03-01,3,1220,1. Crédito ampliado ao setor não financeiro Em...,0.0
3,202205_estm,202205_mensal,202205,mensal,2022-05-01,4,1384,1. Crédito ampliado ao setor não financeiro Em...,0.0
4,202207_estm,202207_mensal,202207,mensal,2022-07-01,4,1391,1. Crédito ampliado ao setor não financeiro Em...,0.0
5,202209_estm,202209_mensal,202209,mensal,2022-09-01,4,1464,1. Crédito ampliado ao setor não financeiro Em...,0.0
6,202210_estm,202210_mensal,202210,mensal,2022-10-01,5,1894,1. Crédito ampliado ao setor não financeiro Em...,0.0
7,202211_estm,202211_mensal,202211,mensal,2022-11-01,4,1536,1. Crédito ampliado ao setor não financeiro Em...,0.0
8,202212_estm,202212_mensal,202212,mensal,2022-12-01,4,1299,1. Crédito ampliado ao setor não financeiro Em...,0.0
9,202301_estm,202301_mensal,202301,mensal,2023-01-01,5,1535,1. Crédito ampliado ao setor não financeiro Em...,0.0


In [17]:
df_paragrafos = processar_raiz("data/atas", anos={2022, 2023, 2024, 2025})
df_docs = montar_docs(df_paragrafos)
df_docs["sentimento"] = df_docs["texto"].apply(indice_sentimento)
df_docs.shape
df_docs

,doc,doc_src,yyyymm,tipo,dt,n_paragrafos,n_palavras,texto,sentimento


In [14]:
df_paragrafos = processar_raiz("data/ref")
df_docs = montar_docs(df_paragrafos)
df_docs["sentimento"] = df_docs["texto"].apply(indice_sentimento)
df_docs.shape
df_docs

,doc,dt,texto,n_paragrafos,n_palavras,tipo,yyyymm,sentimento


# 2.1) Limpando títulos desnecessários

> Seção (markdown) que introduz a **limpeza textual**: remoção de cabeçalhos/legendas, fusão de parágrafos curtos etc.

---
## Bloco 4 — Funções utilitárias de limpeza e fusão


In [4]:
import re
import numpy as np
import pandas as pd

# --- helpers ---
WORD_RE = re.compile(r"\b\w+\b", flags=re.UNICODE)

def count_words(s: str) -> int:
    return len(WORD_RE.findall(s or ""))

def caps_ratio(s: str) -> float:
    toks = re.findall(r"\b[^\W\d_]+\b", s or "", flags=re.UNICODE)
    if not toks: 
        return 0.0
    return sum(t.isupper() for t in toks) / len(toks)

_HEADING_PREFIXES = (
    "nota para a imprensa",
    "sumário","sumario","resumo",
    "apresentação","apresentacao",
    "introdução","introducao",
    "conclusão","conclusoes","conclusao",
)
_CAPTION_HINTS = ("figura","gráfico","grafico","tabela","fonte:")

def is_heading(text: str) -> bool:
    s = (text or "").strip()
    if not s:
        return True
    # "2." ou "2.3." no início
    if re.match(r"^\d+(\.\d+)*\s", s):
        return True
    low = s.lower()
    if low.startswith(_HEADING_PREFIXES):
        return True
    if any(k in low for k in _CAPTION_HINTS):
        return True
    # muito MAIÚSCULO (título), curto e sem pontuação final
    if caps_ratio(s) > 0.6 and len(s.split()) <= 20:
        return True
    if len(s.split()) < 15 and not re.search(r"[.!?;:]\s*$", s):
        return True
    return False

def clean_text_unit(s: str) -> str:
    s = (s or "").replace("\r","")
    s = re.sub(r"[ \t]+", " ", s).strip()
    # remove numeração de seção no início
    s = re.sub(r"^\d+(\.\d+)*\s+", "", s)
    return s

def _fuse_short_paragraphs(df_group: pd.DataFrame, min_words: int = 8, max_merge_span: int = 3) -> pd.DataFrame:
    """Funde parágrafos consecutivos curtos até atingir min_words (no máx. 3 blocos por fusão)."""
    rows, buf, buf_words, buf_span = [], [], 0, 0
    for _, r in df_group.sort_values("pid").iterrows():
        txt = str(r["texto"])
        w = count_words(txt)
        if w >= min_words:
            if buf and buf_words >= min_words:
                rows.append({"texto": " ".join(buf)})
            buf, buf_words, buf_span = [], 0, 0
            rows.append({"texto": txt})
        else:
            buf.append(txt); buf_words += w; buf_span += 1
            if buf_words >= min_words or buf_span >= max_merge_span:
                rows.append({"texto": " ".join(buf)})
                buf, buf_words, buf_span = [], 0, 0
    if buf and buf_words >= min_words:
        rows.append({"texto": " ".join(buf)})
    out = pd.DataFrame(rows)
    out["pid"] = np.arange(1, len(out)+1, dtype=int)
    return out

def limpar_paragrafos(df: pd.DataFrame, min_palavras: int = 8, max_merge_span: int = 3) -> pd.DataFrame:
    """
    Entrada: df_paragrafos com colunas ['doc','dt','pid','texto','n_palavras'].
    Saída: df_limpo com mesmas colunas, mas:
      - headings/legendas removidos
      - parágrafos curtos fundidos
      - mínimo de 8 palavras garantido
      - pid reindexado por (doc, dt)
    """
    # checagem 
    for c in ["doc","dt","pid","texto","n_palavras"]:
        if c not in df.columns:
            raise ValueError(f"Coluna obrigatória ausente: {c}")

    dfx = df.copy()
    dfx["doc"] = dfx["doc"].astype(str)
    dfx["dt"] = pd.to_datetime(dfx["dt"], errors="coerce")
    dfx["pid"] = pd.to_numeric(dfx["pid"], errors="coerce").fillna(0).astype(int)
    dfx["texto"] = dfx["texto"].astype(str)

    # 1) remove vazios/sem letras
    mask_nonempty = dfx["texto"].str.strip().ne("")
    mask_alpha = dfx["texto"].str.contains(r"[A-Za-zÀ-ÖØ-öø-ÿ]", regex=True)
    dfx = dfx[mask_nonempty & mask_alpha].copy()

    # 2) remove headings/legendas
    dfx = dfx[~dfx["texto"].map(is_heading)].copy()

    # 3) limpeza fina + recount
    dfx["texto"] = dfx["texto"].map(clean_text_unit)
    dfx["n_palavras"] = dfx["texto"].map(count_words).astype(int)

    # 4) fusão de curtas por (doc, dt)
    groups = []
    for (doc, dt), g in dfx.sort_values(["doc","pid"]).groupby(["doc","dt"], sort=False):
        g2 = _fuse_short_paragraphs(g[["pid","texto"]], min_words=min_palavras, max_merge_span=max_merge_span)
        g2["doc"], g2["dt"] = doc, dt
        groups.append(g2)
    if groups:
        dfx = pd.concat(groups, ignore_index=True)
    else:
        # nenhum grupo → retorna vazio com colunas padrão
        return dfx.assign(n_palavras=pd.Series(dtype=int))[["doc","dt","pid","texto","n_palavras"]]

    # 5) recount + mínimo final
    dfx["texto"] = dfx["texto"].map(clean_text_unit)
    dfx["n_palavras"] = dfx["texto"].map(count_words).astype(int)
    dfx = dfx[dfx["n_palavras"] >= min_palavras].copy()

    # 6) ordena e reindexa pid por doc
    dfx = dfx.sort_values(["dt","doc","pid"]).reset_index(drop=True)
    dfx["pid"] = dfx.groupby(["doc","dt"]).cumcount() + 1
    return dfx[["doc","dt","pid","texto","n_palavras"]]


**O que faz (principais utilitários):**

* `count_words` — conta **palavras** (tokens) no texto.
* `caps_ratio` — mede proporção de **palavras em MAIÚSCULAS** (útil para detectar títulos).
* `is_heading` — heurística para filtrar **títulos/legendas** (ex.: começa com numeração “2.3”, contém “Figura/Tabela”, alto `caps_ratio`, sem pontuação final).
* `clean_text_unit` — limpeza fina: remove quebras, espaços excessivos, numeração de seção.
* `_fuse_short_paragraphs` — **funde parágrafos muito curtos** (até atingir `min_words`, limitando `max_merge_span`).
* `limpar_paragrafos` — **pipeline** de limpeza:

  1. remove vazios/sem letras,
  2. remove headings/legendas,
  3. limpeza fina + **reconta palavras**,
  4. funde curtos **por (doc, dt)**,
  5. reforça mínimo de palavras,
  6. **reindexa `pid`** (posição do parágrafo no documento/data).

**Conceitos rápidos:**

* **Heurística**: regra prática que funciona “na maioria dos casos”.
* **Reindexar `pid`**: renumerar a ordem dos parágrafos após limpeza/fusão para manter consistência.

**Saída:** funções prontas para uso; nenhuma tabela imediata.

---

## Bloco 5 — Aplicação da limpeza e amostra

In [5]:
# se você já tem df_paragrafos carregado:
df_paragrafos_limpos = limpar_paragrafos(df_paragrafos, min_palavras=8, max_merge_span=3)

# checagem rápida
print(df_paragrafos.shape, "→", df_paragrafos_limpos.shape)
display(df_paragrafos_limpos.head(10))


(1079, 5) → (793, 5)


,doc,dt,pid,texto,n_palavras
0,202201,2022-01-01,1,"Em 2021, o saldo do crédito ampliado ao setor ...",250
1,202201,2022-01-01,2,"O volume de crédito do SFN alcançou R$4,7 tril...",206
2,202201,2022-01-01,3,"O crédito livre às famílias atingiu R$ 1,5 tri...",56
3,202201,2022-01-01,4,"Em 2021, o crédito direcionado atingiu R$ 1, 9...",166
4,202201,2022-01-01,5,"O Indicador de Custo do Crédito (ICC), que med...",93
5,202201,2022-01-01,6,A taxa média de juros das contratações finaliz...,203
6,202201,2022-01-01,7,"A inadimplência do crédito geral atingiu 2, 3%...",59
7,202201,2022-01-01,8,"3. Agregados monetários Em 2021, a base monetá...",363
8,202201,2022-01-01,9,De acordo com a Política de Revisão das Estatí...,85
9,202201,2022-01-01,10,i) Crédito por setor de atividade econômica : ...,101


**O que faz:** aplica a limpeza completa sobre `df_paragrafos`, checa o **efeito na forma** (linhas/colunas) e exibe **amostra** (primeiras 10 linhas).

**Por quê:** validar que títulos/legendas foram removidos, parágrafos curtos foram fundidos e que a estrutura (`doc`, `dt`, `pid`, `texto`, `n_palavras`) permaneceu correta.

**Saída:** `df_paragrafos_limpos` (dataframe limpo).

## Instalação

O que faz: instala libs para carregar modelos (transformers), datasets (datasets), métricas (sklearn) e treino (torch/accelerate).

In [6]:
pip install -U transformers datasets scikit-learn accelerate torch


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from pathlib import Path

# 1) garantir a base de parágrafos
if 'df_paragrafos_limpos' not in globals():
    if Path("paragrafos.csv").exists():
        df_paragrafos_limpos = pd.read_csv("paragrafos.csv")
    else:
        raise RuntimeError("Crie 'df_paragrafos_limpos' ou exporte 'paragrafos.csv' antes.")

# 2) esquema de rótulos de risco
LABELS_RISCO = ["otimista_risco","neutro_risco","pessimista_risco"]

# 3) amostra inicial p/ rotular (ex.: até 900 linhas)
amostra = df_paragrafos_limpos.sample(n=min(900, len(df_paragrafos_limpos)), random_state=42).copy()
amostra["id"] = amostra.apply(lambda r: f"{r['doc']}|{r['dt']}|{r['pid']}", axis=1)

rotular = amostra[["id","doc","dt","pid","texto"]].copy()
rotular["label"] = ""  # você preencherá com um dos LABELS_RISCO

Path("out").mkdir(parents=True, exist_ok=True)
rotular.to_csv("out/dataset_para_rotular_risco.csv", index=False)
print("Criei 'out/dataset_para_rotular_risco.csv'. Preencha 'label' com:", LABELS_RISCO)


Criei 'out/dataset_para_rotular_risco.csv'. Preencha 'label' com: ['otimista_risco', 'neutro_risco', 'pessimista_risco']


In [8]:
import re
import numpy as np
import pandas as pd


path_in = Path("out/dataset_para_rotular_risco.csv")
if not path_in.exists():
    raise RuntimeError("Execute o passo 1 primeiro (gera dataset_para_rotular_risco.csv).")

df = pd.read_csv(path_in)

# NEGATIVO — inadimplência/risco em alta, perdas, stress, rebaixamentos
neg_kw = r"""(?i)\b(
    inadimpl(?:e|ê)n(?:c(?:ia|ial)?|te|tes)?|
    calot(?:e|eiros?)|
    default(?:s)?|
    atras(?:o|os|ou|ar|ados?)|
    provis(?:[õo]es?|[ãa]o|ionamentos?)(?: adicionais?)?|
    npl(?:s)?|
    perd(?:a|as)(?: esperad[as]?| incorrid[as]?)?|
    deterior(?:a(?:ç(?:[ãa]o|ões))?|ou|ar)|
    risco(?:s)?|
    incertez(?:a|as)|
    (?:stress|estresse)|
    liquidez (?:restrit[ao]s?|escass[ao]s?|apertad[ao]s?|curta)|
    pior(?:a(?:ç(?:[ãa]o)?)?|ou)|
    eleva(?:ç(?:[ãa]o|ões)|c(?:a|ã)o|ou)|subid(?:a|as)|subiu|alt(?:a|as)|aument(?:o|os|ou|ar)|
    rebaix(?:amento|ou)s?|downgrade(?:s)?|
    venciment(?:o )?antecipad(?:o|os)|
    (?:quebr(?:a|as)|viola(?:ç(?:[ãa]o)?)|violou) de covenant|covenant breach|
    impairment(?:s)?|
    baixa(?:s)? a preju(?:í|i)zo|write-?off(?:s)?|
    perda por redu(?:ç(?:[ãa]o)) ao valor recuper[aá]vel|
    recupera(?:ç(?:[ãa]o)) judicial|fal[êe]ncia|insolv[êe]ncia|
    execu(?:ç(?:[ãa]o)) de garantias|
    inadimpl(?:e|ê)ncia(?: >? ?90 ?d(?:ias)?)?
)\b"""

# POSITIVO — melhora de crédito, reduções de risco/provisões, upgrades, liquidez forte
pos_kw = r"""(?i)\b(
    melhor(?:a|ou|ar|ias?)|
    redu(?:ç(?:[ãa]o|ões)|ziu|zir)|
    queda(?:s)?|caiu|ca(?:i|í)da(?:s)?|
    diminui(?:u|ç(?:[ãa]o|ões))|
    est[áa]vel(?:es)?|sob controle|controlad(?:o|a|os|as)|
    robust(?:o|a|ez|os|as)|
    solv(?:ê|e)nc(?:ia|e)|
    cobertura (?:maior|superior)|
    provis(?:[õo]es?) (?:menores|liberadas)|revers(?:[ãa]o) de provis(?:[õo]es?)|write-?back(?:s)?|
    npl(?:s)? (?:menor(?:es)?|em queda)|
    inadimpl(?:e|ê)n(?:c(?:ia)?)? (?:menor|em queda|controlad[ao]s?)|
    recupera(?:ç(?:[ãa]o)) de cr[eé]dito(?:s)?|cr[eé]ditos recuperados|
    refor(?:ç(?:o|os)) de garantias|colateral adicional|
    desalavanc(?:e|a)gem|alavancagem menor|
    upgrade(?:s)?|eleva(?:ç(?:[ãa]o)) de rating|perspectiva positiva|
    liquidez (?:s[óo]lida|folgada|confort[aá]vel|robusta)|
    custo do risco (?:menor|em queda)|
    llp(?:s)? (?:menor(?:es)?)|provis(?:[õo]es?) sobre cr[eé]ditos (?:menores)
)\b"""


def score_risco(txt: str) -> float:
    t = txt.lower()
    s = 0
    s += 1.0 * len(re.findall(neg_kw, t))
    s -= 1.0 * len(re.findall(pos_kw, t))
    return float(s)

df["score_heur"] = df["texto"].astype(str).map(score_risco)

# Converter score → sugestão e "confiança"
def sug_conf(s: float):
    if s >= 2:   return "pessimista_risco", min(1.0, 0.6 + 0.1*s)  # mais pessimista, confiança ↑
    if s <= -2:  return "otimista_risco",  min(1.0, 0.6 + 0.1*(-s))
    if -1 <= s <= 1: return "neutro_risco", 0.50
    return ("pessimista_risco" if s>0 else "otimista_risco"), 0.55

out = df.copy()
out["sugestao"], out["conf"] = zip(*out["score_heur"].map(sug_conf))

# Pré-preenche 'label' com a sugestão (você pode filtrar por conf e revisar)
out["label"] = out["sugestao"]

# Ordenar para revisão: casos de menor confiança primeiro
out_rev = out.sort_values("conf", ascending=True)
out_rev


,id,doc,dt,pid,texto,label,score_heur,sugestao,conf
777,202210|2022-10-01 00:00:00|20,202210,2022-10-01,20,As séries estão publicadas no Sistema Gerencia...,neutro_risco,0.0,neutro_risco,0.5
24,202210|2022-10-01 00:00:00|17,202210,2022-10-01,17,"• crédito ampliado ao setor não financeiro, no...",neutro_risco,0.0,neutro_risco,0.5
23,202501|2025-01-01 00:00:00|15,202501,2025-01-01,15,Entre os fluxos mensais dos fatores condiciona...,neutro_risco,0.0,neutro_risco,0.5
22,202401|2024-01-01 00:00:00|16,202401,2024-01-01,16,Entre os fluxos mensais dos fatores condiciona...,neutro_risco,0.0,neutro_risco,0.5
21,202507|2025-07-01 00:00:00|11,202507,2025-07-01,11,"No crédito livre , a taxa média de juros situo...",neutro_risco,1.0,neutro_risco,0.5
...,...,...,...,...,...,...,...,...,...
594,202307|2023-07-01 00:00:00|5,202307,2023-07-01,5,As concessões nominais de crédito totalizaram ...,pessimista_risco,5.0,pessimista_risco,1.0
539,202412|2024-12-01 00:00:00|8,202412,2024-12-01,8,A taxa média de juros das concessões atingiu 2...,pessimista_risco,4.0,pessimista_risco,1.0
571,202412|2024-12-01 00:00:00|4,202412,2024-12-01,4,O estoque das operações de crédito do SFN aume...,pessimista_risco,5.0,pessimista_risco,1.0
92,202307|2023-07-01 00:00:00|13,202307,2023-07-01,13,"O M2 cresceu 1,7% no mês, com saldo total de R...",pessimista_risco,4.0,pessimista_risco,1.0


In [9]:
import pandas as pd
from pathlib import Path

LABELS_RISCO = ["otimista_risco","neutro_risco","pessimista_risco"]

df_risk = out_rev
df_risk["label"] = df_risk["label"].str.strip().str.lower()
df_risk = df_risk[df_risk["label"].isin(LABELS_RISCO)].copy()
df_risk


,id,doc,dt,pid,texto,label,score_heur,sugestao,conf
777,202210|2022-10-01 00:00:00|20,202210,2022-10-01,20,As séries estão publicadas no Sistema Gerencia...,neutro_risco,0.0,neutro_risco,0.5
24,202210|2022-10-01 00:00:00|17,202210,2022-10-01,17,"• crédito ampliado ao setor não financeiro, no...",neutro_risco,0.0,neutro_risco,0.5
23,202501|2025-01-01 00:00:00|15,202501,2025-01-01,15,Entre os fluxos mensais dos fatores condiciona...,neutro_risco,0.0,neutro_risco,0.5
22,202401|2024-01-01 00:00:00|16,202401,2024-01-01,16,Entre os fluxos mensais dos fatores condiciona...,neutro_risco,0.0,neutro_risco,0.5
21,202507|2025-07-01 00:00:00|11,202507,2025-07-01,11,"No crédito livre , a taxa média de juros situo...",neutro_risco,1.0,neutro_risco,0.5
...,...,...,...,...,...,...,...,...,...
594,202307|2023-07-01 00:00:00|5,202307,2023-07-01,5,As concessões nominais de crédito totalizaram ...,pessimista_risco,5.0,pessimista_risco,1.0
539,202412|2024-12-01 00:00:00|8,202412,2024-12-01,8,A taxa média de juros das concessões atingiu 2...,pessimista_risco,4.0,pessimista_risco,1.0
571,202412|2024-12-01 00:00:00|4,202412,2024-12-01,4,O estoque das operações de crédito do SFN aume...,pessimista_risco,5.0,pessimista_risco,1.0
92,202307|2023-07-01 00:00:00|13,202307,2023-07-01,13,"O M2 cresceu 1,7% no mês, com saldo total de R...",pessimista_risco,4.0,pessimista_risco,1.0


# 2) DeB3RTA



In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForMaskedLM.from_pretrained("higopires/DeB3RTa-[base/small]")
tokenizer = AutoTokenizer.from_pretrained("higopires/DeB3RTa-[base/small]")

# Example usage
text = "O mercado financeiro brasileiro apresentou [MASK] no último trimestre."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)


# 3) FinBERT-PT-BR

In [ ]:
from transformers import (
    AutoTokenizer, 
    BertForSequenceClassification,
    pipeline,
)

finbert_pt_br_tokenizer = AutoTokenizer.from_pretrained("lucas-leme/FinBERT-PT-BR")
finbert_pt_br_model = BertForSequenceClassification.from_pretrained("lucas-leme/FinBERT-PT-BR")

finbert_pt_br_pipeline = pipeline(task='text-classification', model=finbert_pt_br_model, tokenizer=finbert_pt_br_tokenizer)
finbert_pt_br_pipeline(['Hoje a bolsa caiu', 'Hoje a bolsa subiu'])